In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig,get_peft_model,PeftModel
from datasets import load_dataset
from dataset.collator import DataCollatorForConMamba
from model import GistLlama
import torch
from dataset.preprocess import Preprocess as Preprocess
class ModelArguments:
    def __init__(self):
      
        self.logging_dir="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/log/llama7b"
        self.tokenizer= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/llama7b-chat"
        self.mtokenizer= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/state-spaces/mamba-2.8btemplate"

        self.modelpath = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/liujiahao12/Model/General/llama-2-7b"
        self.output_dir= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/out/llama7b"
        self.lora=32
args=ModelArguments()

tokenizer=AutoTokenizer.from_pretrained(args.tokenizer)
tokenizer.pad_token_id=0
# model=GistLlama.GistLlama.from_pretrained(args.modelpath)

# if len(tokenizer) == 32000 + 1:
#     pass
# else:
#     # Initialize gist token
#     tokenizer.add_special_tokens({"additional_special_tokens": ["<GIST>"]})
#     model.resize_token_embeddings(len(tokenizer))
#     # Set new word embedding to average of existing word embeddings. For why,
#     # see https://nlp.stanford.edu/~johnhew/vocab-expansion.html
    
#     with torch.no_grad():
            
#                 model.model.embed_tokens.weight[
#                     -1
#                 ] = model.model.embed_tokens.weight[:-1].mean(0)
#                 model.lm_head.weight[-1] = model.lm_head.weight[:-1].mean(0)
          
# gist_token = tokenizer.additional_special_tokens_ids[-1]



/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/miniconda3/envs/ConMamba/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

ImportError: attempted relative import beyond top-level package

In [1]:
import torch
import torch.nn as nn
from peft import LoraConfig
from peft import get_peft_model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.m=nn.Linear(4,6)
        self.n=nn.Linear(4,6)
    def forward(self,x):
        return x
LoRAconfig = LoraConfig(
    target_modules=["n"
    ],
    modules_to_save=["m"],
    r= 2,
    
    )
model=Model()
pmodel=get_peft_model(model,LoRAconfig)



/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/miniconda3/envs/ConMamba/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pmodel.m

ModulesToSaveWrapper(
  (original_module): Linear(in_features=4, out_features=6, bias=True)
  (modules_to_save): ModuleDict(
    (default): Linear(in_features=4, out_features=6, bias=True)
  )
)

In [9]:
from datasets import load_from_disk
d=load_from_disk("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/dataset/conSim")


In [10]:
print("ModulesToSaveWrapper" in str(model.m))

True


In [10]:

tokenizer=AutoTokenizer.from_pretrained(args.tokenizer)
print(tokenizer.pad_token_id)
model.config.pad_token_id

None


0

In [2]:
dp="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/concept/conceptTransformer248/datasets/OpenOrca"
dataset = load_dataset(dp, split="train").select(range(2))
mtokenizer=AutoTokenizer.from_pretrained(args.mtokenizer)
collator=DataCollatorForConMamba(
            tokenizer,
            mtokenizer,
            max_length=256 + 256,  # source=256; target=256
            # Human eval examples are longer.
            max_length_human=384 + 384,  # source=384; target=384
            gist_condition="gist",
            num_gist_tokens=1,
            gist_token=gist_token,
            pad_token=tokenizer.pad_token_id,
            check_correctness=True,
        )
preprocess=Preprocess(tokenizer)

ndataset=dataset.map(preprocess,remove_columns=["system_prompt","question","response","id"])
d=collator(ndataset)
torch.set_printoptions(profile="full")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 2/2 [00:00<00:00, 25.01 examples/s]


In [7]:
tokenizer.save_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama")

('/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama/tokenizer_config.json',
 '/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama/special_tokens_map.json',
 '/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama/tokenizer.model',
 '/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama/added_tokens.json',
 '/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/gistllama/tokenizer.json')

In [6]:
# print(d['attention_mask_gist'][0][0])
# print(d['input_ids'])
print(mtokenizer.decode(d['input_ids'][0]))
a=d['labels'][0]
#get all the non zero elements
a=a[a>=0]

# print(mtokenizer.decode(a))

<s>[INST] <<SYS>>

<</SYS>>

You will be given a definition of a task first, then some input of the task.
This task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.

AFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.
Output: [/INST][
  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],
  ["Ajax Youth Academy", "plays at", "Sportpark De Toekomst"]
]<|endoftext|>


In [2]:
tokenizer=AutoTokenizer.from_pretrained(args.tokenizer)
tokenizer.pad_token_id=0
model=GistLlama.GistLlama.from_pretrained(args.modelpath)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/miniconda3/envs/ConMamba/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it]


In [13]:
import torch
a=torch.randn(3,4)
b=a[0:1,:]
print(b.shape)

torch.Size([1, 4])


In [3]:
if len(tokenizer) == 32000 + 1:
    pass
else:
    # Initialize gist token
    tokenizer.add_special_tokens({"additional_special_tokens": ["<GIST>"]})
    model.resize_token_embeddings(len(tokenizer))
    # Set new word embedding to average of existing word embeddings. For why,
    # see https://nlp.stanford.edu/~johnhew/vocab-expansion.html
    
    with torch.no_grad():
            
                model.model.embed_tokens.weight[
                    -1
                ] = model.model.embed_tokens.weight[:-1].mean(0)
                model.lm_head.weight[-1] = model.lm_head.weight[:-1].mean(0)
          
gist_token = tokenizer.additional_special_tokens_ids[-1]

In [4]:
dp="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/concept/conceptTransformer248/datasets/OpenOrca"
dataset = load_dataset(dp, split="train").select(range(1))

collator=DataCollatorForAlpacaCLM(
            tokenizer,
            # Chosen so that <1% of examples are truncated.
            # See data/alpaca_plus/length_stats.txt for length stats.
            max_length=256 + 256,  # source=256; target=256
            # Human eval examples are longer.
            max_length_human=384 + 384,  # source=384; target=384
            gist_condition="gist",
            num_gist_tokens=1,
            gist_token=gist_token,
            pad_token=tokenizer.pad_token_id,
            check_correctness=True,
        )
preprocess=Preprocess(tokenizer)

In [5]:
ndataset=dataset.map(preprocess,remove_columns=["system_prompt","question","response","id"])
d=collator(ndataset)

Map: 100%|██████████| 1/1 [00:00<00:00,  7.71 examples/s]


In [6]:
torch.set_printoptions(profile="full")
print(d['attention_mask_gist'][0][0])
print(d['input_ids'])
print(tokenizer.decode(d['input_ids'][0]))

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
instance={}
instance['instruction']="hellow"
maybe_gist_str=" ".join(
                    ["<GIST>" for _ in range(2)]
                )
prompt = f"Instruction: {instance['instruction']}\n{maybe_gist_str}\nOutput:" 
print(prompt)

Instruction: hellow
<GIST> <GIST>
Output:


In [ ]:
from model.ConMamba import MambaCache

ModuleNotFoundError: No module named 'model'

In [ ]:
class ModelArguments:
    def __init__(self):
      
        self.logging_dir="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/log/Mamba2.8"
        self.tokenizer= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/state-spaces/mamba-2.8btemplate"
        
        self.modelpath = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/state-spaces/mamba-2.8b-hf"
        self.output_dir= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/out/mamba2.8b"
        self.lora=16
args=ModelArguments()

In [ ]:
LoRAconfig = LoraConfig(
    target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
    
    r=args.lora,
)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(args.modelpath)
pmodel=get_peft_model(model,LoRAconfig)
pmodel=pmodel.half()
pmodel.print_trainable_parameters()

In [ ]:

class ModelArguments:
    def __init__(self):
      
        self.logging_dir="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/log/gemma2b"
        self.tokenizer= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/google/gemma2bchat"
        
        self.modelpath = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/google/gemma2b"
        self.output_dir= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/out/gemma2b"
        self.lora=64
args=ModelArguments()

In [ ]:
class ModelArguments:
    def __init__(self):
      
        self.logging_dir="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/log/llama7b"
        self.tokenizer= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/meta/llama7b-chat"
        
        self.modelpath = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/liujiahao12/Model/General/llama-2-7b"
        self.output_dir= "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/out/llama7b"
        self.lora=32
args=ModelArguments()

In [ ]:
model=AutoModelForCausalLM.from_pretrained(args.modelpath)


In [ ]:
LoRAconfig = LoraConfig(
    target_modules=["q_proj",
    "k_proj",
    ],
    
    r=128
    
    )
pmodel=get_peft_model(model,LoRAconfig)
pmodel=pmodel.half()
pmodel.print_trainable_parameters()

In [ ]:
LoRAconfig = LoraConfig(
    target_modules=["q_proj",
    "k_proj",
    ],
    
    r=args.lora
    
    )

In [ ]:
model=AutoModelForCausalLM.from_pretrained(args.modelpath)
peftpath="/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/ConMamba/out/llama3b/end"
pmodel=PeftModel.from_pretrained(model,peftpath)
pmodel=pmodel.half().to("cuda")
pmodel=pmodel.merge_and_unload()

In [ ]:
template=AutoTokenizer.from_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/concept/conceptTransformer248/param/1btiny1.0")
tokenizer.chat_template=template.chat_template

In [ ]:
tokenizer.save_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/google/gemma2btemplate")

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(args.tokenizer)
pro1="Describe a vivid and unique character, using strong imagery and creative language. Please answer in fewer than two paragraphs."
pro2="What do you think of China? USA and China, which is better?"
pro3="""Premise: A man in a silly hat holding a wand and a toy helicopter sits in a chair.

Hypothesis: A man prepares to entertain an elementary school class.

.Can we conclude that the hypothesis is true if the premise is true?"""

sys3="You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps."
messages1 = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds in the style of a pirate",
        },
        {"role": "user", "content": pro1},
        ]

messages2 = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds in the style of a pirate",
        },
        {"role": "user", "content": pro2},
        ]


messages3 = [
        {
            "role": "system",
            "content": sys3,
        },
        {"role": "user", "content": pro3},
        ]

prompt = tokenizer.apply_chat_template(messages1, tokenize=False, add_generation_prompt=True)
print(prompt)
# inputs=tokenizer(prompt,return_tensors='pt',add_special_tokens=False).to("cuda")
# o=pmodel.generate(inputs.input_ids,max_new_tokens=800,do_sample=True)
# print(tokenizer.decode(o[0]))

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds in the style of a pirate
<</SYS>>

Describe a vivid and unique character, using strong imagery and creative language. Please answer in fewer than two paragraphs. [/INST]


In [ ]:
import torch 
a=torch.rand(3,4)
for i in range(a.shape[0]):
    h=a[i]
    print(h[0])
print(a)

tensor(0.2131)
tensor(0.8084)
tensor(0.1227)
tensor([[0.2131, 0.3198, 0.0153, 0.6657],
        [0.8084, 0.4656, 0.2772, 0.0239],
        [0.1227, 0.2124, 0.7831, 0.6805]])


In [ ]:
model=AutoModelForCausalLM.from_pretrained(args.modelpath)
pmodel=get_peft_model(model,LoRAconfig)
pmodel=pmodel.half()
pmodel.print_trainable_parameters()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/state-spaces/mamba-2.8b-hf")
template=AutoTokenizer.from_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/openlm-research/open_llama_3b_v2")
tinytemplate=AutoTokenizer.from_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/concept/conceptTransformer248/param/1btiny1.0")
tokenizer.chat_template=tinytemplate.chat_template
template.chat_template=tinytemplate.chat_template

In [ ]:
tokenizer.save_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/state-spaces/mamba-2.8btemplate")

In [ ]:
template.save_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/yuandl/LLMs/openlm-research/open_llama_3b_v2template")

In [ ]:
systemprompt="You are a helpful AI assistant"
question="How are you"
messages = [
        {
            "role": "system",
            "content": systemprompt,
        },
        {"role": "user", "content": question},
        ]
tem = template.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)

In [ ]:
print(tem)



In [ ]:
prompt=template.encode(tem,add_special_tokens=True)
print(prompt)